In [44]:
import pickle
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import operator

from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score, recall_score, precision_score
from bayes_opt import BayesianOptimization

from general_functions import create_balanced_dataset

In [45]:
file = open("dataset/zone_4.pickle", "rb")
zone_4 = pickle.load(file)

file = open("dataset/zone_7.pickle", "rb")
zone_7 = pickle.load(file)
file = None

In [46]:
#zone_7_resampled = create_balanced_dataset(["dataset/zone_7.pickle"])

#with open("dataset/zone_7_resampled.pickle", "wb") as file:
#    pickle.dump(zone_7_resampled, file)

In [47]:
with open("dataset/zone_4_resampled.pickle", "rb") as file:
    zone_4_resampled = pickle.load(file)
with open("dataset/zone_7_resampled.pickle", "rb") as file:
    zone_7_resampled = pickle.load(file)

In [48]:
zone_4.shape

(7852140, 82)

In [49]:
experiment_arr = [(zone_4_resampled, zone_7), (zone_7_resampled, zone_4)]

In [7]:
most_important_features = zone_4.columns.tolist()[1:]
for num_features in [81, 50, 35, 25, 20, 15, 10]:
    
    most_important_features = most_important_features[:num_features]
    print(f"Features used in experiment:\n{most_important_features}")
    print("\n")
    feature_importances = {i:0 for i in most_important_features}
    y_test_all = []
    pred_all = []
    
    
    for (training_dataset, test_dataset) in experiment_arr:

        X_train = training_dataset.filter(items=most_important_features).loc[:, training_dataset.filter(items=most_important_features).columns != "label_3m"]
        y_train = training_dataset["label_3m"]
        training_dataset = None

        _max_features = 25 if num_features > 25 else num_features
        clf = RandomForestClassifier(n_estimators=200, class_weight="balanced", max_features=_max_features, n_jobs=-1)

        clf.fit(X_train, y_train)

        X_test = test_dataset.filter(items=most_important_features).loc[:, test_dataset.filter(items=most_important_features).columns != "label_3m"]
        y_test = test_dataset["label_3m"]
        
        print(f"Amount of features X_train: {len(X_train.columns)}, X_test: {len(X_test.columns)}")

        pred = clf.predict(X_test)

        for i, pred_var in enumerate(pred):
            y_test_all.append(y_test[i])
            pred_all.append(pred_var)

        importances = clf.feature_importances_

        
        for i, importance in enumerate(importances):
            feature_importances[most_important_features[i]] += importance
        
    for importance_name in most_important_features:
        feature_importances[importance_name] /= 2
    
    
    
    most_important_features = list(dict(sorted(feature_importances.items(),
                                               key=operator.itemgetter(1),reverse=True)).keys())

    print("\n")
    print(f"Importances for experiment:")
    for key in most_important_features:
        print(key, " - ", feature_importances[key])
    print("\n")
    print(f"Amount of features used:   {num_features}")
    print("Cohen's kappa score        ", cohen_kappa_score(y_test_all, pred_all))
    print("Accuracy score             ", accuracy_score(y_test_all, pred_all))
    print("Recall score               ", recall_score(y_test_all, pred_all))
    print("Precision score            ", precision_score(y_test_all, pred_all))
    print("\n------------------------------------------------------\n")

Features used in experiment:
['hpmf_raw', 'skyview_raw', 'impundment_raw', 'slope_raw', 'DEM_ditch_detection', 'DEM_ditch_detection_no_streams', 'conic_mean', 'skyview_non_ditch', 'skyview_gabor', 'conic_mean_no_streams', 'skyview_gabor_no_streams', 'skyview_mean_2', 'skyview_mean_3', 'skyview_mean_4', 'skyview_mean_6', 'skyview_median_2', 'skyview_median_4', 'skyview_median_6', 'skyview_min_2', 'skyview_min_4', 'skyview_min_6', 'skyview_max_2', 'skyview_max_4', 'skyview_max_6', 'skyview_std_2', 'skyview_std_4', 'skyview_std_6', 'impundment_amplified', 'impoundment_amplified_no_streams', 'impundment_mean_2', 'impundment_mean_3', 'impundment_mean_4', 'impundment_mean_6', 'impundment_median_2', 'impundment_median_4', 'impundment_median_6', 'impundment_min_2', 'impundment_min_4', 'impundment_min_6', 'impundment_max_2', 'impundment_max_4', 'impundment_max_6', 'impundment_std_2', 'impundment_std_4', 'impundment_std_6', 'hpmf_filter', 'hpmf_gabor', 'hpmf_gabor_no_streams', 'hpmf_filter_no_st

Cohen's kappa score         0.4287659646094656
Accuracy score              0.9587795174309169
Recall score                0.7124483846470013
Precision score             0.3257753874914897

------------------------------------------------------

Features used in experiment:
['impundment_mean_3', 'impundment_mean_4', 'impundment_median_4', 'impundment_mean_2', 'impundment_amplified', 'impoundment_amplified_no_streams', 'hpmf_median_4', 'skyview_max_6', 'hpmf_mean_4', 'skyview_gabor', 'skyview_non_ditch', 'slope_min_6', 'skyview_max_4', 'skyview_gabor_no_streams', 'slope_median_6', 'impundment_max_6', 'hpmf_filter', 'skyview_median_6', 'slope_non_ditch', 'impundment_std_4', 'slope_std_6', 'hpmf_filter_no_streams', 'impundment_median_6', 'hpmf_mean_6', 'slope_min_4', 'hpmf_mean_3', 'impundment_mean_6', 'slope_min_2', 'slope_mean_6', 'skyview_max_2', 'slope_std_4', 'impundment_median_2', 'hpmf_std_6', 'skyview_min_6', 'impundment_std_6']


Amount of features X_train: 35, X_test: 35
Amount o

Cohen's kappa score         0.3320598002814307
Accuracy score              0.941501934504479
Recall score                0.6889087895127947
Precision score             0.23933660717578606

------------------------------------------------------



In [50]:
most_important_features = zone_4.columns.tolist()[1:]
zone_4 = None
zone_7 = None
zone_4_resampled = None
zone_7_resampled = None
for num_features in [81, 65, 62, 59, 56, 53, 50, 47, 44, 41, 38, 35]:
    
    most_important_features = most_important_features[:num_features]
    print(f"Features used in experiment:\n{most_important_features}")
    print("\n")
    feature_importances = {i:0 for i in most_important_features}
    y_test_all = []
    pred_all = []
    
    
    for (training_dataset, test_dataset) in experiment_arr:

        X_train = training_dataset.filter(items=most_important_features).loc[:, training_dataset.filter(items=most_important_features).columns != "label_3m"]
        y_train = training_dataset["label_3m"]
        training_dataset = None

        _max_features = 25 if num_features > 25 else num_features
        clf = RandomForestClassifier(n_estimators=200, class_weight="balanced", max_features=_max_features, n_jobs=-1)

        clf.fit(X_train, y_train)

        X_test = test_dataset.filter(items=most_important_features).loc[:, test_dataset.filter(items=most_important_features).columns != "label_3m"]
        y_test = test_dataset["label_3m"]
        
        print(f"Amount of features X_train: {len(X_train.columns)}, X_test: {len(X_test.columns)}")

        pred = clf.predict(X_test)

        for i, pred_var in enumerate(pred):
            y_test_all.append(y_test[i])
            pred_all.append(pred_var)

        importances = clf.feature_importances_

        
        for i, importance in enumerate(importances):
            feature_importances[most_important_features[i]] += importance
        
    for importance_name in most_important_features:
        feature_importances[importance_name] /= 2
    
    
    
    most_important_features = list(dict(sorted(feature_importances.items(),
                                               key=operator.itemgetter(1),reverse=True)).keys())

    print("\n")
    print(f"Importances for experiment:")
    for key in most_important_features:
        print(key, " - ", feature_importances[key])
    print("\n")
    print(f"Amount of features used:   {num_features}")
    print("Cohen's kappa score        ", cohen_kappa_score(y_test_all, pred_all))
    print("Accuracy score             ", accuracy_score(y_test_all, pred_all))
    print("Recall score               ", recall_score(y_test_all, pred_all))
    print("Precision score            ", precision_score(y_test_all, pred_all))
    print("\n------------------------------------------------------\n")

Features used in experiment:
['hpmf_raw', 'skyview_raw', 'impundment_raw', 'slope_raw', 'DEM_ditch_detection', 'DEM_ditch_detection_no_streams', 'conic_mean', 'skyview_non_ditch', 'skyview_gabor', 'conic_mean_no_streams', 'skyview_gabor_no_streams', 'skyview_mean_2', 'skyview_mean_3', 'skyview_mean_4', 'skyview_mean_6', 'skyview_median_2', 'skyview_median_4', 'skyview_median_6', 'skyview_min_2', 'skyview_min_4', 'skyview_min_6', 'skyview_max_2', 'skyview_max_4', 'skyview_max_6', 'skyview_std_2', 'skyview_std_4', 'skyview_std_6', 'impundment_amplified', 'impoundment_amplified_no_streams', 'impundment_mean_2', 'impundment_mean_3', 'impundment_mean_4', 'impundment_mean_6', 'impundment_median_2', 'impundment_median_4', 'impundment_median_6', 'impundment_min_2', 'impundment_min_4', 'impundment_min_6', 'impundment_max_2', 'impundment_max_4', 'impundment_max_6', 'impundment_std_2', 'impundment_std_4', 'impundment_std_6', 'hpmf_filter', 'hpmf_gabor', 'hpmf_gabor_no_streams', 'hpmf_filter_no_st

Cohen's kappa score         0.42758002488020674
Accuracy score              0.9584068801626053
Recall score                0.7157283582983334
Precision score             0.32394397430574634

------------------------------------------------------

Features used in experiment:
['impundment_mean_3', 'impundment_mean_4', 'impundment_median_4', 'impundment_mean_2', 'hpmf_mean_4', 'impundment_amplified', 'impoundment_amplified_no_streams', 'hpmf_median_4', 'skyview_max_6', 'hpmf_mean_3', 'skyview_gabor', 'skyview_non_ditch', 'slope_min_6', 'skyview_max_4', 'skyview_gabor_no_streams', 'impundment_max_2', 'impundment_median_2', 'impundment_std_4', 'impundment_max_6', 'slope_non_ditch', 'hpmf_filter', 'hpmf_min_2', 'hpmf_filter_no_streams', 'impundment_mean_6', 'impundment_median_6', 'hpmf_mean_6', 'impundment_std_6', 'slope_std_6', 'slope_min_4', 'slope_median_6', 'skyview_median_6', 'skyview_min_6', 'impundment_max_4', 'hpmf_min_4', 'skyview_max_2', 'slope_mean_6', 'hpmf_gabor_no_streams', 's

Amount of features X_train: 56, X_test: 56
Amount of features X_train: 56, X_test: 56


Importances for experiment:
impundment_mean_3  -  0.15724250428216957
impundment_mean_4  -  0.08180407803189814
impundment_median_4  -  0.0622845124549371
impundment_mean_2  -  0.047266316461828876
impundment_amplified  -  0.027051403706817544
hpmf_mean_4  -  0.026849187788144643
impoundment_amplified_no_streams  -  0.023458540661683062
hpmf_median_4  -  0.020255672890071262
skyview_max_6  -  0.019431705768611824
skyview_gabor  -  0.016912709508749763
skyview_non_ditch  -  0.016792733485545275
slope_min_6  -  0.015266900367479585
skyview_max_4  -  0.014875177133376506
skyview_gabor_no_streams  -  0.014457425343139273
impundment_std_4  -  0.014262227339428203
impundment_max_6  -  0.013402964341934848
slope_non_ditch  -  0.013251726497364126
hpmf_filter  -  0.013005088332447741
hpmf_mean_3  -  0.012933113757200514
hpmf_filter_no_streams  -  0.012584455105503902
hpmf_mean_6  -  0.012474571090879037
imp

Cohen's kappa score         0.4283547633162076
Accuracy score              0.9587223992440277
Recall score                0.7123041202457394
Precision score             0.3253890687715424

------------------------------------------------------

Features used in experiment:
['impundment_mean_3', 'impundment_mean_4', 'impundment_median_4', 'impundment_mean_2', 'impundment_amplified', 'impoundment_amplified_no_streams', 'hpmf_median_4', 'skyview_max_6', 'hpmf_mean_4', 'skyview_non_ditch', 'skyview_gabor', 'slope_min_6', 'skyview_max_4', 'hpmf_mean_3', 'skyview_gabor_no_streams', 'impundment_std_4', 'impundment_max_6', 'slope_non_ditch', 'hpmf_filter', 'hpmf_mean_6', 'hpmf_filter_no_streams', 'slope_std_6', 'impundment_median_6', 'skyview_median_6', 'impundment_median_2', 'slope_median_6', 'slope_min_4', 'impundment_mean_6', 'impundment_max_2', 'hpmf_mean_2', 'hpmf_std_6', 'skyview_min_6', 'slope_mean_6', 'impundment_std_6', 'skyview_std_6', 'skyview_max_2', 'slope_min_2', 'hpmf_min_2', 'h

Cohen's kappa score         0.42389556100738757
Accuracy score              0.9581493707447906
Recall score                0.7098271276957708
Precision score             0.3213886495926212

------------------------------------------------------

Features used in experiment:
['impundment_mean_3', 'impundment_mean_4', 'impundment_median_4', 'impundment_mean_2', 'impundment_amplified', 'hpmf_median_4', 'impoundment_amplified_no_streams', 'skyview_max_6', 'skyview_non_ditch', 'skyview_gabor', 'slope_min_6', 'skyview_max_4', 'hpmf_mean_6', 'hpmf_mean_4', 'impundment_max_6', 'skyview_gabor_no_streams', 'slope_median_6', 'hpmf_filter', 'impundment_std_4', 'impundment_median_6', 'slope_std_6', 'hpmf_filter_no_streams', 'slope_non_ditch', 'skyview_median_6', 'skyview_min_6', 'hpmf_std_6', 'slope_min_4', 'hpmf_mean_3', 'slope_mean_6', 'impundment_mean_6', 'skyview_std_6', 'hpmf_min_4', 'impundment_median_2', 'slope_std_4', 'impundment_std_6', 'slope_min_2', 'slope_median_2', 'skyview_max_2']


A

In [64]:
from itertools import product

In [65]:
params = [["gini", "entropy"],
          [35, None],
          ['log2', 'sqrt'],
          [2, 10],
          ["balanced", "balanced_subsample", None],
          [100, 200, 300]]

In [66]:
most_important_features = ['impundment_mean_3', 'impundment_mean_4', 'impundment_median_4', 'impundment_mean_2',
                           'impundment_amplified', 'hpmf_mean_4', 'impoundment_amplified_no_streams', 'hpmf_median_4',
                           'skyview_max_6', 'skyview_gabor', 'skyview_non_ditch', 'slope_min_6', 'skyview_max_4',
                           'skyview_gabor_no_streams', 'impundment_std_4', 'impundment_max_6', 'slope_non_ditch',
                           'hpmf_filter', 'hpmf_mean_3', 'hpmf_filter_no_streams', 'hpmf_mean_6', 'impundment_median_6',
                           'impundment_median_2', 'slope_std_6', 'impundment_mean_6', 'slope_median_6', 'slope_min_4',
                           'hpmf_min_2', 'impundment_std_6', 'skyview_median_6', 'skyview_min_6', 'slope_mean_6',
                           'impundment_max_2', 'skyview_std_6', 'slope_min_2', 'skyview_max_2', 'hpmf_min_4',
                           'impundment_max_4', 'slope_std_4', 'hpmf_gabor', 'hpmf_std_6', 'slope_max_4',
                           'hpmf_gabor_no_streams', 'skyview_min_4', 'slope_median_4', 'hpmf_mean_2', 'slope_median_2',
                           'slope_std_2', 'hpmf_min_6', 'hpmf_max_6', 'skyview_mean_6', 'skyview_median_4', 'hpmf_std_4']

In [70]:
param_combos = list(product(*params))

In [71]:
param_combos

[('gini', 35, 'log2', 2, 'balanced', 100),
 ('gini', 35, 'log2', 2, 'balanced', 200),
 ('gini', 35, 'log2', 2, 'balanced', 300),
 ('gini', 35, 'log2', 2, 'balanced_subsample', 100),
 ('gini', 35, 'log2', 2, 'balanced_subsample', 200),
 ('gini', 35, 'log2', 2, 'balanced_subsample', 300),
 ('gini', 35, 'log2', 2, None, 100),
 ('gini', 35, 'log2', 2, None, 200),
 ('gini', 35, 'log2', 2, None, 300),
 ('gini', 35, 'log2', 10, 'balanced', 100),
 ('gini', 35, 'log2', 10, 'balanced', 200),
 ('gini', 35, 'log2', 10, 'balanced', 300),
 ('gini', 35, 'log2', 10, 'balanced_subsample', 100),
 ('gini', 35, 'log2', 10, 'balanced_subsample', 200),
 ('gini', 35, 'log2', 10, 'balanced_subsample', 300),
 ('gini', 35, 'log2', 10, None, 100),
 ('gini', 35, 'log2', 10, None, 200),
 ('gini', 35, 'log2', 10, None, 300),
 ('gini', 35, 'sqrt', 2, 'balanced', 100),
 ('gini', 35, 'sqrt', 2, 'balanced', 200),
 ('gini', 35, 'sqrt', 2, 'balanced', 300),
 ('gini', 35, 'sqrt', 2, 'balanced_subsample', 100),
 ('gini', 3

In [ ]:
params_and_score = []

for combo in param_combos:
    y_test_all = []
    pred_all = []
    
    params = {'criterion': combo[0],
    'max_depth': combo[1],
    'max_features': combo[2],
    'min_samples_split': combo[3],
    'class_weight': combo[4],
    'n_estimators': combo[5]}
    print(f"params: {params}")
    
    for (training_dataset, test_dataset) in experiment_arr:

        X_train = training_dataset.filter(items=most_important_features).loc[:, training_dataset.filter(items=most_important_features).columns != "label_3m"]
        y_train = training_dataset["label_3m"]
        training_dataset = None

        clf = RandomForestClassifier(**params, n_jobs=-1)
        print(clf)

        clf.fit(X_train, y_train)

        X_test = test_dataset.filter(items=most_important_features).loc[:, test_dataset.filter(items=most_important_features).columns != "label_3m"]
        y_test = test_dataset["label_3m"]
        
        print(f"Amount of features X_train: {len(X_train.columns)}, X_test: {len(X_test.columns)}")

        pred = clf.predict(X_test)

        for i, pred_var in enumerate(pred):
            y_test_all.append(y_test[i])
            pred_all.append(pred_var)

    print("\n")
    print(f"Amount of features used:   {num_features}")
    print("Cohen's kappa score        ", cohen_kappa_score(y_test_all, pred_all))
    print("Accuracy score             ", accuracy_score(y_test_all, pred_all))
    print("Recall score               ", recall_score(y_test_all, pred_all))
    print("Precision score            ", precision_score(y_test_all, pred_all))
    print("\n------------------------------------------------------\n")
    params_and_score.append((params, cohen_kappa_score(y_test_all, pred_all)))

print(params_and_score)

params: {'criterion': 'gini', 'max_depth': 35, 'max_features': 'log2', 'min_samples_split': 2, 'class_weight': 'balanced', 'n_estimators': 100}
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=35, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)
